In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('resume_data.csv')

In [3]:
print(df.columns)

Index(['address', 'career_objective', 'skills', 'educational_institution_name',
       'degree_names', 'passing_years', 'educational_results', 'result_types',
       'major_field_of_studies', 'professional_company_names', 'company_urls',
       'start_dates', 'end_dates', 'related_skils_in_job', 'positions',
       'locations', 'responsibilities', 'extra_curricular_activity_types',
       'extra_curricular_organization_names',
       'extra_curricular_organization_links', 'role_positions', 'languages',
       'proficiency_levels', 'certification_providers', 'certification_skills',
       'online_links', 'issue_dates', 'expiry_dates', '﻿job_position_name',
       'educationaL_requirements', 'experiencere_requirement',
       'age_requirement', 'responsibilities.1', 'skills_required',
       'matched_score'],
      dtype='object')


In [11]:
columns = df.columns.drop(['career_objective', 'responsibilities.1'])
for column in columns:
    print(f'{column}: {df[column].dtype} : {df[column].nunique()}')
    #print(f'{df[column].unique()}')
    for unique in df[column].unique():
        print(f'{unique}: {df[df[column] == unique].shape[0]}')
    print('-' * 50)

address: object : 28
nan: 0
Sunnyvale, CA: 28
Trenton, NJ: 28
10 King's College Road, Rm. 3302 Toronto, Ontario, M5S 3G4 Canada: 28
City, State Zip Code: 28
139 Alder Drive, Mississaugua, Ontario, L5N 6P1: 28
Portland, OR: 28
Cambridge, MA: 28
Greer, SC: 28
1234 Thundar Lane S, Fargo, ND 58102: 28
Euclid, OH: 28
Vancouver, BC: 28
Computer Science Department University of Kentucky Lexington, KY 40506-0046: 28
160 Bleecker Street, Apartment 4DW New York, NY 10012: 28
Westlake, TX: 28
P.O BOX 2200 ELDORET: 28
1255 University of Utah Circle, Las Vegas, NV 84112: 28
2383 Suwanee Pointe Drive, Lawrenceville, GA 30043: 28
Virginia Beach, VA: 28
321 Sesame Street, Boca Raton, FL 33829: 28
Street name, City, YO1 5DD: 28
Chicago, IL: 28
Pittsburgh, PA: 28
Denver, CO: 28
Hanover, NH: 28
Hoboken, NJ: 28
Department of Computer Science, University of British Columbia, 2366 Main Mall, Vancouver, BC, V6T 1Z4, Canada: 28
Malvern, PA: 28
Address, City, Province, Country: 28
-----------------------------

In [12]:
import nltk
from nltk.corpus import wordnet as wn
from itertools import product

# Ensure you have the WordNet data
nltk.download('wordnet')
nltk.download('punkt')

def sentence_similarity(sentence1, sentence2):
    # Tokenize sentences
    words1 = nltk.word_tokenize(sentence1)
    words2 = nltk.word_tokenize(sentence2)
    
    # Get synsets for each word in the sentences
    synsets1 = [wn.synsets(word) for word in words1]
    synsets2 = [wn.synsets(word) for word in words2]
    
    # Filter out empty synsets
    synsets1 = [synset[0] for synset in synsets1 if synset]
    synsets2 = [synset[0] for synset in synsets2 if synset]
    
    # Compute similarity between all synset pairs
    similarities = []
    for synset1, synset2 in product(synsets1, synsets2):
        similarity = synset1.path_similarity(synset2)
        if similarity is not None:
            similarities.append(similarity)
    
    # Average the similarities
    if similarities:
        return sum(similarities) / len(similarities)
    else:
        return 0
    
# Example sentences
sentence1 = "Technical Support Troubleshooting Collaboration"
sentence2 = "Machine Learning Leadership Cross-Functional Collaboration"

# Compute sentence similarity
similarity = sentence_similarity(sentence1, sentence2)
print(f"Sentence similarity: {similarity}")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Sentence similarity: 0.16806414072039072
